In [0]:
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
root_path = '/content/drive/My Drive/DSCI 303 Final Project/Data_Trimming_and_TFIDF_Models/'

In [0]:
with open(root_path + 'preprocessed_500k_reviews.json') as json_file: # Open json file to load
    d = json.load(json_file) # Load the data as a string
    data = json.loads(d) #convert to list of dictionaries

In [0]:
df = pd.DataFrame(data) # Convert list of dictionaries {review, stars} to dataframe

In [9]:
df.head(2)

,review,stars
0,total bill horribl servic 8g crook actual ner...,1.0
1,ador travi hard rock new kelli cardena salon ...,5.0


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = df['review']
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(corpus)

In [12]:
vectorizer.get_feature_names()

['00015m',
 '00a',
 '00am',
 '00at',
 '00buck',
 '00cad',
 '00coupon',
 '00dollar',
 '00ea',
 '00h30',
 '00ish',
 '00manigur',
 '00noon',
 '00p',
 '00plu',
 '00pm',
 '00pmin',
 '00small',
 '00the',
 '00to',
 '00u00',
 '00which',
 '01167jbted',
 '0140am',
 '01am',
 '01pm',
 '02am',
 '02p',
 '02pm',
 '0300p',
 '03a',
 '03am',
 '03p',
 '03pm',
 '0433eciexx6namw',
 '04am',
 '04apr17',
 '04nov',
 '04pm',
 '04th',
 '05am',
 '05aug2018',
 '05lb',
 '05p',
 '05pm',
 '06am',
 '06pm',
 '07am',
 '07gh',
 '07pm',
 '08am',
 '08pm',
 '0936am',
 '09am',
 '09awnfv',
 '09pm',
 '09th',
 '09u8',
 '0a',
 '0awoj6zm',
 '0c',
 '0crunch',
 '0ct',
 '0d',
 '0eajijhhcrdugiava',
 '0f',
 '0h',
 '0huv',
 '0jhfnovwvjpm5l1nhc',
 '0l',
 '0lace',
 '0lcnaclqzri',
 '0ldtown',
 '0lrlxb6zajma59o8dfyp',
 '0lztz6afnpjte2aoscfdda',
 '0mbeg',
 '0mg',
 '0mph',
 '0n',
 '0ne',
 '0nli',
 '0npryzelwwo',
 '0nqg3mo8irajzkozhjnvsg',
 '0oz',
 '0p',
 '0pi',
 '0q7shgsjczq',
 '0s',
 '0sandwich',
 '0star',
 '0t',
 '0thi',
 '0unc',
 '0ur',
 

In [13]:
X_train_tfidf.shape

(500000, 128911)

In [0]:
from sklearn.model_selection import train_test_split, cross_val_score
x_train, x_test, y_train, y_test = train_test_split(X_train_tfidf,df['stars'],test_size=0.2, random_state = 101)

In [0]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(x_train, y_train)

In [16]:
#Testing sklearn naive bayes classifier with our tfidf on our training data to test accuracy 
predicted = clf.predict(x_test)
np.mean(predicted == y_test) 

0.5697

In [17]:
from sklearn.linear_model import SGDClassifier
SVM_clf = SGDClassifier(max_iter=1000, tol=1e-3)
SVM_clf.fit(x_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [18]:
#Testing sklearn SVM classifier with our tfidf on our training data to test accuracy 
predicted = SVM_clf.predict(x_test)
np.mean(predicted == y_test) 

0.62247

In [0]:
#random forest cross-validation
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators = 5) #5 trees
scores = cross_val_score(rfc, x_train, y_train, cv = 5)

In [0]:
scores

array([0.49812516, 0.49979168, 0.50208333, 0.50052086, 0.49868734])

In [0]:
#tuning the hyperparameters
for n_trees in np.arange(10,160,20):
  rfc = RandomForestClassifier(n_estimators = n_trees)
  rfc_cv_scores = cross_val_score(rfc, x_train, y_train, cv=8)
  print("CV with {} trees:".format(n_trees))
  print(rfc_cv_scores)

CV with 10 trees:
[0.5440304  0.54666667 0.542      0.54773333 0.5402     0.54046667
 0.54066667 0.55017001]
CV with 30 trees:
[0.57302846 0.5714     0.57166667 0.57653333 0.57133333 0.5698
 0.57606667 0.5735049 ]
CV with 50 trees:
[0.57496167 0.5794     0.577      0.58073333 0.57706667 0.57533333
 0.57526667 0.58210547]
CV with 70 trees:
[0.57696154 0.57966667 0.5766     0.58093333 0.57346667 0.57606667
 0.57806667 0.57823855]
CV with 90 trees:
[0.57376175 0.5788     0.57766667 0.58086667 0.576      0.57373333
 0.57833333 0.57590506]
CV with 110 trees:
[0.57816146 0.57826667 0.57586667 0.57686667 0.57486667 0.57373333
 0.57673333 0.57683846]
CV with 130 trees:
[0.57396174 0.5782     0.57653333 0.57893333 0.57393333 0.576
 0.57706667 0.57863858]


KeyboardInterrupt: ignored

In [0]:
rfc.fit(x_train, y_train)
rfc.predict(x_test)
np.mean(predicted == y_test)

0.4992333333333333